In [1]:
import os

In [2]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=3, suppress=True)

pd.options.display.float_format = lambda num: f'{num:.4f}'

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={"figure.figsize": (12, 8)})

In [4]:
from utils.ingest import read_data
from fixed_effects import within_group
from gmm import lagged_gmm

In [5]:
from utils.printing import np_to_pmatrix

#### Constants

In [6]:
data_path = "data/hw3.xls"

### Data inject

In [7]:
data = read_data(data_path)

### 2. Homogenous panel

#### (a)

The panel is:
- homogenous
- no heterogeneity in the slope parameters
- no endogeneity
- no cross-sectional dependence

Given the assumptions, the suitable estimator is the Withing Group (or Fixed Effects) estimators.

In [8]:
regs = ["d(lnY)", "INF"]
beta, fixed_effects, resid, var, dw = within_group(data, "S/Y", regs, lags = 0)


Withing regression:  

--- β:

d(lnY)   0.293056
INF      0.179817, 


--- Standard errors:

         d(lnY)      INF
d(lnY) 0.017580 0.000416
INF    0.000416 0.003499
        
--- Durbin-Watson
0.5910  

--- Pesaran
4.4644  p=0.0000



#### 3. Dynamic homogenous panels

Within regressor with lagged value

In [9]:
regs = ["d(lnY)", "INF"]
beta, fixed_effects, resid, var, dw = within_group(data, "S/Y", regs, lags = 1, title="Lagged Within regression")


Lagged Within regression:  

--- β:

d(lnY)      0.023697
INF         0.021070
lag_1_S/Y   0.848010, 


--- Standard errors:

             d(lnY)       INF  lag_1_S/Y
d(lnY)     0.001158  0.000059  -0.000196
INF        0.000059  0.000318  -0.000157
lag_1_S/Y -0.000196 -0.000157   0.001078
        
--- Durbin-Watson
1.3085  

--- Pesaran
0.1213  p=0.4517



In [10]:
with open("data/cv_matrix.tex", "w") as file:
    
    file.write(np_to_pmatrix(beta, prec = 4))

#### 4. Enogeneity in homogenous dynamic panel

In [11]:
lags = 2

In [12]:
_ = lagged_gmm(
        data, "S/Y",
        regressors=["d(lnY)", "INF"], lag_inst=lags
    )



Withing regression:  

--- β:

d(lnY)      -1.331277
INF         -0.459295
lag_1_S/Y    3.493394, 


--- Standard errors:

              d(lnY)       INF  lag_1_S/Y
d(lnY)      5.459203  2.315545 -13.229239
INF         2.315545  1.009244  -5.721096
lag_1_S/Y -13.229239 -5.721096  32.702864
        
--- Durbin-Watson
1.8486  



In [13]:
_ = lagged_gmm(
        data, "S/Y",
        regressors=["d(lnY)", "INF"], lag_inst=lags,
        gmm=True
    )


Withing regression:  

--- β:

d(lnY)      -1.907793
INF         -0.710240
lag_1_S/Y    4.909449, 


--- Standard errors:

              d(lnY)        INF  lag_1_S/Y
d(lnY)     12.341023   5.237213 -29.920110
INF         5.237213   2.285950 -12.951252
lag_1_S/Y -29.920110 -12.951252  74.020684
        
--- Overidentifying Restrictions
0.3887  p=0.5330

--- Durbin-Watson
1.8422  



#### 5. Cross-sectional dependence in a homogeneous dynamic panel.

In [14]:
from ccep import ccep

In [15]:
beta, resid_by_n, cov = ccep(
        data, "S/Y", ["d(lnY)", "INF"],  title="CCEP estimator")


CCEP estimator:  

--- β:

d(lnY)      0.112180
INF         0.050982
lag_1_S/Y   0.733041, 


--- Standard errors:

             d(lnY)       INF  lag_1_S/Y
d(lnY)     0.047324  0.006842  -0.007022
INF        0.006842  0.018163  -0.003740
lag_1_S/Y -0.007022 -0.003740   0.027735
        


#### 6. Endogeneity

In [16]:
_ = lagged_gmm(
        data, "S/Y",
        regressors=["SG/Y"], lag_inst=2, title="IV estimation of SG/Y -> S/Y"
    )

_ = lagged_gmm(
        data, "S/Y",
        regressors=["SG/Y"], lag_inst=2, title="GMM estimation of SG/Y -> S/Y",
        gmm=True, is_lagged_instrumented=True
    )


IV estimation of SG/Y -> S/Y:  

--- β:

SG/Y        -0.190944
lag_1_S/Y    0.541782, 


--- Standard errors:

              SG/Y  lag_1_S/Y
SG/Y      0.004458   0.012474
lag_1_S/Y 0.012474   0.042449
        
--- Durbin-Watson
1.7009  


GMM estimation of SG/Y -> S/Y:  

--- β:

SG/Y        -0.122965
lag_1_S/Y    0.802800, 


--- Standard errors:

              SG/Y  lag_1_S/Y
SG/Y      0.000644   0.000330
lag_1_S/Y 0.000330   0.001818
        
--- Overidentifying Restrictions
1.5297  p=0.4654

--- Durbin-Watson
1.7923  

